In [1]:
import requests
import pandas as pd
import json
from pandas import DataFrame
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd


In [2]:
data = pd.read_csv("~/Documents/jtnp_routesAndDesc_joined.csv")

In [3]:
data

,Unnamed: 0,index,id,name,type,rating,stars,starVotes,pitches,location,url,imgSqSmall,imgSmall,imgSmallMed,imgMedium,longitude,latitude,Description
0,0,0,105725464,JBMFP,Boulder,V5,4.9,124,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10572546...,https://cdn-files.apstatic.com/climb/110869832...,https://cdn-files.apstatic.com/climb/110869832...,https://cdn-files.apstatic.com/climb/110869832...,https://cdn-files.apstatic.com/climb/110869832...,-116.1654,34.0148,This classic problem takes a proud line direct...
1,1,1,105722992,Pigpen,Boulder,V4,4.9,96,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10572299...,https://cdn-files.apstatic.com/climb/116126755...,https://cdn-files.apstatic.com/climb/116126755...,https://cdn-files.apstatic.com/climb/116126755...,https://cdn-files.apstatic.com/climb/116126755...,-116.1581,34.0154,Behind Cyclops rock is this awesome boulder pr...
2,2,2,105722755,Slashface,Boulder,V3 R,4.9,79,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10572275...,https://cdn-files.apstatic.com/climb/106344662...,https://cdn-files.apstatic.com/climb/106344662...,https://cdn-files.apstatic.com/climb/106344662...,https://cdn-files.apstatic.com/climb/106344662...,-116.0870,33.9534,Another of Joshua Tree's finest boulder proble...
3,3,3,105722350,Gunsmoke,Boulder,V3,4.6,280,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10572235...,https://cdn-files.apstatic.com/climb/112179623...,https://cdn-files.apstatic.com/climb/112179623...,https://cdn-files.apstatic.com/climb/112179623...,https://cdn-files.apstatic.com/climb/112179623...,-116.1446,34.0280,"Classic. Like Yosemite's Midnight Lightning, ..."
4,4,4,105722608,Stem Gem,Boulder,V4,4.7,107,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10572260...,https://cdn-files.apstatic.com/climb/106341989...,https://cdn-files.apstatic.com/climb/106341989...,https://cdn-files.apstatic.com/climb/106341989...,https://cdn-files.apstatic.com/climb/106341989...,-116.1636,34.0184,Located in a small group of campsites on the S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364,1364,312,109595290,Unnamed .8,Boulder,V-easy,0.0,0,NaN,"{California,""San Jacinto Mountains"",""The Tramw...",https://www.mountainproject.com/route/10959529...,NaN,NaN,NaN,NaN,-116.6349,33.8236,Climb the slabby face and arete. Also one poss...
1365,1365,313,107785167,Tenderfoot,Boulder,V-easy,0.0,0,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10778516...,https://cdn-files.apstatic.com/climb/107785171...,https://cdn-files.apstatic.com/climb/107785171...,https://cdn-files.apstatic.com/climb/107785171...,https://cdn-files.apstatic.com/climb/107785171...,-116.1585,34.0933,Just climb up the sloping side of the rock.
1366,1366,314,107103905,Sunlight Line Center,Boulder,V-easy,0.0,0,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10710390...,NaN,NaN,NaN,NaN,-116.0872,33.9664,"Unkown V? (yes, not actually V-easy). My gues..."
1367,1367,315,107103915,Sunlight Line Right,Boulder,V-easy,0.0,0,NaN,"{California,""Joshua Tree National Park"",""*Josh...",https://www.mountainproject.com/route/10710391...,NaN,NaN,NaN,NaN,-116.0872,33.9664,"Unkown V? (yes, not actually V-easy). My gues..."


In [10]:
desc = data['Description']

In [32]:
desc[3]

"Classic.  Like Yosemite's Midnight Lightning, Hueco Tank's Ghetto Simulator or Boulder's Monkey Traverse, Gunsmoke requires little introduction.   The Gunsmoke Traverse is L-shaped and is usually done from left to right.  Start as far left as possible, move right along the wall for about 50' past 2 crux sections, turn the corner (third crux), and continue another 35' along rails to a final bulge.  If your guns aren't too smoked, reverse your moves back to the start.     GUNSMOKE IS LIKELY THE SINGLE MOST CROWDED ROUTE IN THE PARK, and it's not uncommon to find a dozen or more people there at any given time.  Several other quality problems can be found in the vicinity, including the classic stemming problem Streetcar Named Desire (V6) and The Chube (V2).  Southern exposure makes this a hot one during mid-day - best done in the late afternoon. "

In [34]:
import nltk
from nltk.corpus import stopwords

rawText = desc[3]

In [35]:
rawText

"Classic.  Like Yosemite's Midnight Lightning, Hueco Tank's Ghetto Simulator or Boulder's Monkey Traverse, Gunsmoke requires little introduction.   The Gunsmoke Traverse is L-shaped and is usually done from left to right.  Start as far left as possible, move right along the wall for about 50' past 2 crux sections, turn the corner (third crux), and continue another 35' along rails to a final bulge.  If your guns aren't too smoked, reverse your moves back to the start.     GUNSMOKE IS LIKELY THE SINGLE MOST CROWDED ROUTE IN THE PARK, and it's not uncommon to find a dozen or more people there at any given time.  Several other quality problems can be found in the vicinity, including the classic stemming problem Streetcar Named Desire (V6) and The Chube (V2).  Southern exposure makes this a hot one during mid-day - best done in the late afternoon. "

In [36]:
# breaking up the raw text into sentences
sentText = nltk.sent_tokenize(rawText)
print(sentText)

['Classic.', "Like Yosemite's Midnight Lightning, Hueco Tank's Ghetto Simulator or Boulder's Monkey Traverse, Gunsmoke requires little introduction.", 'The Gunsmoke Traverse is L-shaped and is usually done from left to right.', "Start as far left as possible, move right along the wall for about 50' past 2 crux sections, turn the corner (third crux), and continue another 35' along rails to a final bulge.", "If your guns aren't too smoked, reverse your moves back to the start.", "GUNSMOKE IS LIKELY THE SINGLE MOST CROWDED ROUTE IN THE PARK, and it's not uncommon to find a dozen or more people there at any given time.", 'Several other quality problems can be found in the vicinity, including the classic stemming problem Streetcar Named Desire (V6) and The Chube (V2).', 'Southern exposure makes this a hot one during mid-day - best done in the late afternoon.']


In [41]:
# breaking up sentences into words
for i in sentText:
    print(nltk.word_tokenize(i))


['Classic', '.']
['Like', 'Yosemite', "'s", 'Midnight', 'Lightning', ',', 'Hueco', 'Tank', "'s", 'Ghetto', 'Simulator', 'or', 'Boulder', "'s", 'Monkey', 'Traverse', ',', 'Gunsmoke', 'requires', 'little', 'introduction', '.']
['The', 'Gunsmoke', 'Traverse', 'is', 'L-shaped', 'and', 'is', 'usually', 'done', 'from', 'left', 'to', 'right', '.']
['Start', 'as', 'far', 'left', 'as', 'possible', ',', 'move', 'right', 'along', 'the', 'wall', 'for', 'about', '50', "'", 'past', '2', 'crux', 'sections', ',', 'turn', 'the', 'corner', '(', 'third', 'crux', ')', ',', 'and', 'continue', 'another', '35', "'", 'along', 'rails', 'to', 'a', 'final', 'bulge', '.']
['If', 'your', 'guns', 'are', "n't", 'too', 'smoked', ',', 'reverse', 'your', 'moves', 'back', 'to', 'the', 'start', '.']
['GUNSMOKE', 'IS', 'LIKELY', 'THE', 'SINGLE', 'MOST', 'CROWDED', 'ROUTE', 'IN', 'THE', 'PARK', ',', 'and', 'it', "'s", 'not', 'uncommon', 'to', 'find', 'a', 'dozen', 'or', 'more', 'people', 'there', 'at', 'any', 'given', 'tim

In [42]:
# breaking up the raw text into words
wordText = nltk.word_tokenize(rawText)
wordText

['Classic',
 '.',
 'Like',
 'Yosemite',
 "'s",
 'Midnight',
 'Lightning',
 ',',
 'Hueco',
 'Tank',
 "'s",
 'Ghetto',
 'Simulator',
 'or',
 'Boulder',
 "'s",
 'Monkey',
 'Traverse',
 ',',
 'Gunsmoke',
 'requires',
 'little',
 'introduction',
 '.',
 'The',
 'Gunsmoke',
 'Traverse',
 'is',
 'L-shaped',
 'and',
 'is',
 'usually',
 'done',
 'from',
 'left',
 'to',
 'right',
 '.',
 'Start',
 'as',
 'far',
 'left',
 'as',
 'possible',
 ',',
 'move',
 'right',
 'along',
 'the',
 'wall',
 'for',
 'about',
 '50',
 "'",
 'past',
 '2',
 'crux',
 'sections',
 ',',
 'turn',
 'the',
 'corner',
 '(',
 'third',
 'crux',
 ')',
 ',',
 'and',
 'continue',
 'another',
 '35',
 "'",
 'along',
 'rails',
 'to',
 'a',
 'final',
 'bulge',
 '.',
 'If',
 'your',
 'guns',
 'are',
 "n't",
 'too',
 'smoked',
 ',',
 'reverse',
 'your',
 'moves',
 'back',
 'to',
 'the',
 'start',
 '.',
 'GUNSMOKE',
 'IS',
 'LIKELY',
 'THE',
 'SINGLE',
 'MOST',
 'CROWDED',
 'ROUTE',
 'IN',
 'THE',
 'PARK',
 ',',
 'and',
 'it',
 "'s",
 '

In [43]:
# word frequency before removing punctuations and stop words
print('Before text processing')
wordFreqBefore = nltk.FreqDist(wordText)
for iWord in wordFreqBefore.most_common(30):
    print('%-15s\t%6d' % iWord)

Before text processing
.              	     8
,              	     8
the            	     6
's             	     4
and            	     4
to             	     4
(              	     3
)              	     3
a              	     3
or             	     2
Traverse       	     2
Gunsmoke       	     2
The            	     2
is             	     2
done           	     2
left           	     2
right          	     2
as             	     2
along          	     2
'              	     2
crux           	     2
your           	     2
THE            	     2
in             	     2
Classic        	     1
Like           	     1
Yosemite       	     1
Midnight       	     1
Lightning      	     1
Hueco          	     1


In [45]:
# removing punctuations and stopwords
wordDePunct = [w.lower() for w in wordText if w.isalpha()]
stop_words = set(stopwords.words('english'))  # stop words in English
wordNoStopwd = [w for w in wordDePunct if w not in stop_words]


# word frequency after removing punctuations and stop words
print('After text processing')
wordFreqAfter = nltk.FreqDist(wordNoStopwd)
for iWord in wordFreqAfter.most_common(100):
    print('%-15s\t%6d' % iWord)

After text processing
gunsmoke       	     3
classic        	     2
traverse       	     2
done           	     2
left           	     2
right          	     2
start          	     2
along          	     2
crux           	     2
like           	     1
yosemite       	     1
midnight       	     1
lightning      	     1
hueco          	     1
tank           	     1
ghetto         	     1
simulator      	     1
boulder        	     1
monkey         	     1
requires       	     1
little         	     1
introduction   	     1
usually        	     1
far            	     1
possible       	     1
move           	     1
wall           	     1
past           	     1
sections       	     1
turn           	     1
corner         	     1
third          	     1
continue       	     1
another        	     1
rails          	     1
final          	     1
bulge          	     1
guns           	     1
smoked         	     1
reverse        	     1
moves          	     1
back           	     1
likely      

In [47]:
wordlist1 = ['dyno','smear','crimp','undercling','traverse']
wordlist1

['dyno', 'smear', 'crimp', 'undercling', 'traverse']